# Download counts for nteract

In [ ]:
import IPython.display
import pandas as pd
import requests

In [ ]:
data = requests.get("https://api.github.com/repos/nteract/nteract/releases").json()

In [ ]:
data[0]

The releases API only gives us the filename, so we convert a release asset URL like `nteract-darwin-x64.zip` into a platform key like `darwin-x64`.

In [ ]:
def strip_off_release(browser_download_url):
    filename = browser_download_url.split("/")[-1]
    basename = filename.split(".")[0]
    system = basename.split("-")[1:]
    return "-".join(system)

In [ ]:
releases = [
    {
        "version": release["tag_name"],
        "counts": {
            strip_off_release(asset["browser_download_url"]): asset["download_count"]
            for asset in release["assets"]
        },
    }
    for release in data
]
releases[:2]

In [ ]:
versions = []
frames = []

for release in releases:
    versions.append(release["version"])
    frames.append(pd.DataFrame.from_dict(release["counts"], orient="index"))

df = pd.concat(frames, keys=versions).reset_index()
df.columns = ["version", "os", "count"]
df["os"] = df.os.replace("os-x", "darwin-x64")
df.head()

In [ ]:
from distutils.version import LooseVersion

versions = sorted(set(df.version.values.tolist()), key=LooseVersion)
versions[:5]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = "retina"

with sns.color_palette("colorblind", len(versions)):
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(1, 1, 1)
    ax = sns.barplot(x="version", y="count", hue="os", data=df, order=versions)
    ax.set_xticklabels(versions, rotation=30)
    ax.set(xlabel="Version", ylabel="Count")
    plt.legend(bbox_to_anchor=(1, 1), loc=2, borderaxespad=0)
    plt.show()